# 第三题：实现决策树

实验内容：  
使用LendingClub Safe Loans数据集：
1. 实现信息增益、信息增益率、基尼指数三种划分标准
2. 使用给定的训练集完成三种决策树的训练过程
3. 计算三种决策树在最大深度为10时在训练集和测试集上的精度，查准率，查全率，F1值

在这部分，我们会实现一个很简单的二叉决策树

## 1. 读取数据

In [3]:
# 导入类库
import pandas as pd
import numpy as np
import json

In [4]:
# 导入数据
loans = pd.read_csv('data/lendingclub/lending-club-data.csv', low_memory=False)
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


数据中有两列是我们想预测的指标，一项是safe_loans，一项是bad_loans，分别表示正例和负例，我们对其进行处理，将正例的safe_loans设为1，负例设为-1，删除bad_loans这列

In [5]:
# 对数据进行预处理，将safe_loans作为标记
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']

我们只使用grade, term, home_ownership, emp_length这四列作为特征，safe_loans作为标记，只保留loans中的这五列

In [6]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

查看前五行数据

In [7]:
loans.head()
print(loans['safe_loans'].value_counts())

 1    99457
-1    23150
Name: safe_loans, dtype: int64


## 2. 划分训练集和测试集

In [8]:
from sklearn.utils import shuffle
loans = shuffle(loans, random_state = 34)

split_line = int(len(loans) * 0.6)
train_data = loans.iloc[: split_line]
test_data = loans.iloc[split_line:]

## 3. 特征预处理

可以看到所有的特征都是离散类型的特征，需要对数据进行预处理，使用one-hot编码对其进行处理。

one-hot编码的思想就是将离散特征变成向量，假设特征$A$有三种取值$\{a, b, c\}$，这三种取值等价，如果我们使用1,2,3三个数字表示这三种取值，那么在计算时就会产生偏差，有一些涉及距离度量的算法会认为，2和1离得近，3和1离得远，但这三个值应该是等价的，这种表示方法会造成模型在判断上出现偏差。解决方案就是使用一个三维向量表示他们，用$[1, 0, 0]$表示a，$[0, 1, 0]$表示b，$[0, 0, 1]$表示c，这样三个向量之间的距离就都是相等的了，任意两个向量在欧式空间的距离都是$\sqrt{2}$。这就是one-hot编码是思想。

pandas中使用get_dummies生成one-hot向量

In [9]:
def one_hot_encoding(data, features_categorical):
    '''
    Parameter
    ----------
    data: pd.DataFrame
    
    features_categorical: list(str)
    '''
    
    # 对所有的离散特征遍历
    for cat in features_categorical:
        
        # 对这列进行one-hot编码，前缀为这个变量名
        one_encoding = pd.get_dummies(data[cat], prefix = cat)
        
        # 将生成的one-hot编码与之前的dataframe拼接起来
        data = pd.concat([data, one_encoding],axis=1)
        
        # 删除掉原始的这列离散特征
        del data[cat]
    
    return data

首先对训练集生成one-hot向量，然后对测试集生成one-hot向量，这里需要注意的是，如果训练集中，特征$A$的取值为$\{a, b, c\}$，这样我们生成的特征就有三列，分别为$A\_a$, $A\_b$, $A\_c$，然后我们使用这个训练集训练模型，模型就就会考虑这三个特征，在测试集中如果有一个样本的特征$A$的值为$d$，那它的$A\_a$，$A\_b$，$A\_c$就都为0，我们不去考虑$A\_d$，因为这个特征在训练模型的时候是不存在的。

In [10]:
train_data = one_hot_encoding(train_data, features)

In [11]:
train_data.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
84320,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
121308,-1,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
58376,-1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
66430,1,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
65344,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


获取所有特征的名字

In [12]:
train_data['emp_length_5 years'].value_counts()

0    67854
1     5710
Name: emp_length_5 years, dtype: int64

In [13]:
one_hot_features = train_data.columns.tolist()
one_hot_features.remove(target)
one_hot_features

['grade_A',
 'grade_B',
 'grade_C',
 'grade_D',
 'grade_E',
 'grade_F',
 'grade_G',
 'term_ 36 months',
 'term_ 60 months',
 'home_ownership_MORTGAGE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'emp_length_1 year',
 'emp_length_10+ years',
 'emp_length_2 years',
 'emp_length_3 years',
 'emp_length_4 years',
 'emp_length_5 years',
 'emp_length_6 years',
 'emp_length_7 years',
 'emp_length_8 years',
 'emp_length_9 years',
 'emp_length_< 1 year']

接下来是对测试集进行one_hot编码，但只要保留出现在one_hot_features中的特征即可·

In [14]:
test_data_tmp = one_hot_encoding(test_data, features)

In [15]:
# 创建一个空的DataFrame
test_data = pd.DataFrame(columns = train_data.columns)
for feature in train_data.columns:
    # 如果训练集中当前特征在test_data_tmp中出现了，将其复制到test_data中
    if feature in test_data_tmp.columns:
        test_data[feature] = test_data_tmp[feature].copy()
    else:
        # 否则就用全为0的列去替代
        test_data[feature] = np.zeros(test_data_tmp.shape[0], dtype = 'uint8')

In [16]:
test_data.head()

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
37225,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
101585,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
31865,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
97692,1,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
88181,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [17]:
train_data.shape

(73564, 25)

In [18]:
test_data.shape

(49043, 25)

训练集有37224个样本，测试集有9284个样本，**处理完后，所有的特征都是0和1，标记是1和-1**，以上就是数据预处理流程

## 4. 实现3种特征划分准则

决策树中有很多常用的特征划分方法，比如信息增益、信息增益率、基尼指数

我们需要实现一个函数，它的作用是，给定决策树的某个结点内的所有样本的标记，让它计算出对应划分指标的值是多少

接下来我们会实现上述三种划分指标

**这里我们约定，将所有特征取值为0的样本，划分到左子树，特征取值为1的样本，划分到右子树**

### 4.1 信息增益
信息熵：
$$
\mathrm{Ent}(D) = - \sum^{\vert \mathcal{Y} \vert}_{k = 1} p_k \mathrm{log}_2 p_k
$$

信息增益：
$$
\mathrm{Gain}(D, a) = \mathrm{Ent}(D) - \sum^{V}_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Ent}(D^v)
$$

计算信息熵时约定：若$p = 0$，则$p \log_2p = 0$

**下面的函数需要填写两个部分**

In [19]:
def information_entropy(labels_in_node):
    '''
    求当前结点的信息熵
    
    Parameter
    ----------
    labels_in_node: np.ndarray, 如[-1, 1, -1, 1, 1]
    
    Returns
    ----------
    float: information entropy
    '''
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出标记为1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出标记为-1的个数
    num_of_negative = len(labels_in_node[labels_in_node == -1])                           # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive / num_of_samples
    
    # 统计负例的概率
    prob_negative = num_of_negative / num_of_samples                                      # YOUR CODE HERE
    
    if prob_positive == 0:
        positive_part = 0
    else:
        positive_part = prob_positive * np.log2(prob_positive)
    
    if prob_negative == 0:
        negative_part = 0
    else:
        negative_part = prob_negative * np.log2(prob_negative)
    
    return - ( positive_part + negative_part )

下面是6个测试样例

In [20]:
# 信息熵测试样例1
example_labels = np.array([-1, -1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.97095

# 信息熵测试样例2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
print(information_entropy(example_labels)) # 0.86312
    
# 信息熵测试样例3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
print(information_entropy(example_labels)) # 0.86312

# 信息熵测试样例4
example_labels = np.array([-1] * 9 + [1] * 8)
print(information_entropy(example_labels)) # 0.99750

# 信息熵测试样例5
example_labels = np.array([1] * 8)
print(information_entropy(example_labels)) # 0

# 信息熵测试样例6
example_labels = np.array([])
print(information_entropy(example_labels)) # 0

0.9709505944546686
0.863120568566631
0.863120568566631
0.9975025463691152
-0.0
0


接下来完成计算所有特征的信息增益的函数  
**需要填写三个部分**

In [21]:
def compute_information_gains(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益
    
    Parameter
    ----------
        data: pd.DataFrame，传入的样本，带有特征和标记的dataframe
        
        features: list(str)，特征名组成的list
        
        target: str, 标记(label)的名字
        
        annotate, boolean，是否打印所有特征的信息增益值，默认为False
        
    Returns
    ----------
        information_gains: dict, key: str, 特征名
                                 value: float，信息增益
    '''
    
    # 我们将每个特征划分的信息增益值存储在一个dict中
    # 键是特征名，值是信息增益值
    information_gains = dict()
    
    # 对所有的特征进行遍历，使用信息增益对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy = information_entropy(right_split_target)                          # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = len(right_split_target) / (len(left_split_target) + len(right_split_target))  # YOUR CODE HERE
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
            
        # 计算使用当前特征划分的信息增益
        gain = current_entropy - right_weight*right_entropy - left_weight*left_entropy    # YOUR CODE HERE
        
        # 将特征名与增益值以键值对的形式存储在information_gains中
        information_gains[feature] = gain
        
        if annotate:
            print(" ", feature, gain)
            
    return information_gains

In [22]:
# 信息增益测试样例1
print(compute_information_gains(train_data, one_hot_features, target)['grade_A']) # 0.01759

# 信息增益测试样例2
print(compute_information_gains(train_data, one_hot_features, target)['term_ 60 months']) # 0.01429

# 信息增益测试样例3
print(compute_information_gains(train_data, one_hot_features, target)['grade_B']) # 0.00370

0.017591980178887745
0.014291850329400169
0.0037049200345330435


### 4.2 信息增益率
信息增益率：

$$
\mathrm{Gain\_ratio}(D, a) = \frac{\mathrm{Gain}(D, a)}{\mathrm{IV}(a)}
$$

其中

$$
\mathrm{IV}(a) = - \sum^V_{v=1} \frac{\vert D^v \vert}{\vert D \vert} \log_2 \frac{\vert D^v \vert}{\vert D \vert}
$$

完成计算所有特征信息增益率的函数  
**这里要完成五个部分**

In [23]:
def compute_information_gain_ratios(data, features, target, annotate = False):
    '''
    计算所有特征的信息增益率并保存起来
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gain_ratios: dict, key: str, 特征名
                       value: float，信息增益率
    '''
    
    gain_ratios = dict()
    
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target =  data[data[feature] == 1][target]
            
        # 计算左子树的信息熵
        left_entropy = information_entropy(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

        # 计算右子树的信息熵
        right_entropy = information_entropy(right_split_target)                             # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight =  len(right_split_target) / (len(left_split_target) + len(right_split_target)) # YOUR CODE HERE
        
        # 计算当前结点的信息熵
        current_entropy = information_entropy(data[target])
        
        # 计算当前结点的信息增益
        
        gain = current_entropy - right_weight*right_entropy - left_weight*left_entropy      # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为0的值
        if left_weight == 0:
            left_IV = 0
        else:
            left_IV = left_weight*np.log2(left_weight)                                     # YOUR CODE HERE
        
        # 计算IV公式中，当前特征为1的值
        if right_weight == 0:
            right_IV = 0
        else:
            right_IV = right_weight*np.log2(right_weight)                                  # YOUR CODE HERE
        
        # IV 等于所有子树IV之和的相反数
        IV = - (left_IV + right_IV)
            
        # 计算使用当前特征划分的信息增益率
        # 这里为了防止IV是0，导致除法得到np.inf（无穷），在分母加了一个很小的小数
        gain_ratio = gain / (IV + np.finfo(np.longdouble).eps)
        
        # 信息增益率的存储
        gain_ratios[feature] = gain_ratio
        
        if annotate:
            print(" ", feature, gain_ratio)
            
    return gain_ratios

In [24]:
# 信息增益率测试样例1
print(compute_information_gain_ratios(train_data, one_hot_features, target)['grade_A']) # 0.02573

# 信息增益率测试样例2
print(compute_information_gain_ratios(train_data, one_hot_features, target)['grade_B']) # 0.00417

# 信息增益率测试样例3
print(compute_information_gain_ratios(train_data, one_hot_features, target)['term_ 60 months']) # 0.01970

0.025734780668006163288
0.004175495069430480813
0.01970936271862220511


### 4.3 基尼指数
数据集$D$的基尼值：

$$
\begin{aligned}
\mathrm{Gini}(D) & = \sum^{\vert \mathcal{Y} \vert}_{k=1} \sum_{k' \neq k} p_k p_{k'}\\
& = 1 - \sum^{\vert \mathcal{Y} \vert}_{k=1} p^2_k.
\end{aligned}
$$

属性$a$的基尼指数：

$$
\mathrm{Gini\_index}(D, a) = \sum^V_{v = 1} \frac{\vert D^v \vert}{\vert D \vert} \mathrm{Gini}(D^v)
$$

完成数据集基尼值的计算  
**这里需要填写三部分**

In [25]:
def gini(labels_in_node):
    '''
    计算一个结点内样本的基尼指数
    
    Paramters
    ----------
    label_in_data: np.ndarray, 样本的标记，如[-1, -1, 1, 1, 1]
    
    Returns
    ---------
    gini: float，基尼指数
    '''
    
    # 统计样本总个数
    num_of_samples = labels_in_node.shape[0]
    
    if num_of_samples == 0:
        return 0
    
    # 统计出1的个数
    num_of_positive = len(labels_in_node[labels_in_node == 1])
    
    # 统计出-1的个数
    num_of_negative = len(labels_in_node[labels_in_node == -1])         # YOUR CODE HERE
    
    # 统计正例的概率
    prob_positive = num_of_positive / num_of_samples
    
    # 统计负例的概率
    prob_negative = num_of_negative / num_of_samples                    # YOUR CODE HERE
     
    # 计算基尼值
    gini =  1 - prob_negative**2 - prob_positive**2                           # YOUR CODE HERE
    
    return gini

In [26]:
# 基尼值测试样例1
example_labels = np.array([-1, -1, 1, 1, 1])
print(gini(example_labels)) # 0.48

# 基尼值测试样例2
example_labels = np.array([-1, -1, 1, 1, 1, 1, 1])
print(gini(example_labels)) # 0.40816
    
# 基尼值测试样例3
example_labels = np.array([-1, -1, -1, -1, -1, 1, 1])
print(gini(example_labels)) # 0.40816

# 基尼值测试样例4
example_labels = np.array([-1] * 9 + [1] * 8)
print(gini(example_labels)) # 0.49827

# 基尼值测试样例5
example_labels = np.array([1] * 8)
print(gini(example_labels)) # 0

# 基尼值测试样例6
example_labels = np.array([])
print(gini(example_labels)) # 0

0.48
0.40816326530612246
0.40816326530612246
0.49826989619377154
0.0
0


然后计算所有特征的基尼指数  
**这里需要填写三部分**

In [27]:
def compute_gini_indices(data, features, target, annotate = False):
    '''
    计算使用各个特征进行划分时，各特征的基尼指数
    
    Parameter
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    gini_indices: dict, key: str, 特征名
                       value: float，基尼指数
    '''
    
    gini_indices = dict()
    # 对所有的特征进行遍历，使用当前的划分方法对每个特征进行计算
    for feature in features:
        # 左子树保证所有的样本的这个特征取值为0
        left_split_target = data[data[feature] == 0][target]
        
        # 右子树保证所有的样本的这个特征取值为1
        right_split_target = data[data[feature] == 1][target]
            
        # 计算左子树的基尼值
        left_gini = gini(left_split_target)
        
        # 计算左子树的权重
        left_weight = len(left_split_target) / (len(left_split_target) + len(right_split_target))

        # 计算右子树的基尼值
        right_gini = gini(right_split_target)                                      # YOUR CODE HERE
        
        # 计算右子树的权重
        right_weight = len(right_split_target) / (len(left_split_target) + len(right_split_target))   # YOUR CODE HERE
        
        # 计算当前结点的基尼指数
        gini_index =  right_weight*right_gini + left_weight*left_gini              # YOUR CODE HERE
        
        # 存储
        gini_indices[feature] = gini_index
        
        if annotate:
            print(" ", feature, gini_index)
            
    return gini_indices

In [28]:
# 基尼指数测试样例1
print(compute_gini_indices(train_data, one_hot_features, target)['grade_A']) # 0.30095

# 基尼指数测试样例2
print(compute_gini_indices(train_data, one_hot_features, target)['grade_B']) # 0.30568

# 基尼指数测试样例3
print(compute_gini_indices(train_data, one_hot_features, target)['term_ 36 months']) # 0.30055

0.3009520964964362
0.3056855375882364
0.30055418611740065


## 5. 完成最优特征的选择 

到此，我们完成了三种划分策略的实现，接下来就是完成获取最优特征的函数  
**这里需要填写三个部分**

In [29]:
def best_splitting_feature(data, features, target, criterion = 'gini', annotate = False):
    '''
    给定划分方法和数据，找到最优的划分特征
    
    Parameters
    ----------
    data: pd.DataFrame, 带有特征和标记的数据
    
    features: list(str)，特征名组成的list
    
    target: str， 特征的名字
    
    criterion: str, 使用哪种指标，三种选项: 'information_gain', 'gain_ratio', 'gini'
    
    annotate: boolean, default False，是否打印注释
    
    Returns
    ----------
    best_feature: str, 最佳的划分特征的名字
    
    '''
    if criterion == 'information_gain':
        if annotate:
            print('using information gain')
        
        # 得到当前所有特征的信息增益
        information_gains = compute_information_gains(data, features, target, annotate)
    
        # information_gains是一个dict类型的对象，我们要找值最大的那个元素的键是谁
        # 根据这些特征和他们的信息增益，找到最佳的划分特征
        best_feature = max(information_gains, key=information_gains.get)                   # YOUR CODE HERE
        
        return best_feature

    elif criterion == 'gain_ratio':
        if annotate:
            print('using information gain ratio')
        
        # 得到当前所有特征的信息增益率
        gain_ratios = compute_information_gain_ratios(data, features, target, annotate)
    
        # 根据这些特征和他们的信息增益率，找到最佳的划分特征
        best_feature = max(gain_ratios, key=gain_ratios.get)                               # YOUR CODE HERE

        return best_feature
    
    elif criterion == 'gini':
        if annotate:
            print('using gini')
        
        # 得到当前所有特征的基尼指数
        gini_indices = compute_gini_indices(data, features, target, annotate)
        
        # 根据这些特征和他们的基尼指数，找到最佳的划分特征
        best_feature = max(gini_indices, key=gini_indices.get)                             # YOUR CODE HERE

        return best_feature
    else:
        raise Exception("传入的criterion不合规!", criterion)

## 6. 判断结点内样本的类别是否为同一类

**这里需要填写两个部分**

In [30]:
def intermediate_node_num_mistakes(labels_in_node):
    '''
    求树的结点中，样本数少的那个类的样本有多少，比如输入是[1, 1, -1, -1, 1]，返回2
    
    Parameter
    ----------
    labels_in_node: np.ndarray, pd.Series
    
    Returns
    ----------
    int：个数
    
    '''
    # 如果传入的array为空，返回0
    if len(labels_in_node) == 0:
        return 0
    
    # 统计1的个数
    num_of_one =  len(labels_in_node[labels_in_node==1])                              # YOUR CODE HERE
    
    # 统计-1的个数
    num_of_minus_one = len(labels_in_node[labels_in_node==-1])                        # YOUR CODE HERE
    
    return num_of_one if num_of_minus_one > num_of_one else num_of_minus_one

In [31]:
# 测试样例1
print(intermediate_node_num_mistakes(np.array([1, 1, -1, -1, -1]))) # 2

# 测试样例2
print(intermediate_node_num_mistakes(np.array([]))) # 0

# 测试样例3
print(intermediate_node_num_mistakes(np.array([1]))) # 0

2
0
0


## 7. 创建叶子结点

In [32]:
def create_leaf(target_values):
    '''
    计算出当前叶子结点的标记是什么，并且将叶子结点信息保存在一个dict中
    
    Parameter:
    ----------
    target_values: pd.Series, 当前叶子结点内样本的标记

    Returns:
    ----------
    leaf: dict，表示一个叶结点，
            leaf['splitting_features'], None，叶结点不需要划分特征
            leaf['left'], None，叶结点没有左子树
            leaf['right'], None，叶结点没有右子树
            leaf['is_leaf'], True, 是否是叶子结点
            leaf['prediction'], int, 表示该叶子结点的预测值
    '''
    # 创建叶子结点
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True}
   
    # 数结点内-1和+1的个数
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # 叶子结点的标记使用少数服从多数的原则，为样本数多的那类的标记，保存在 leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = 1
    else:
        leaf['prediction'] = -1

    # 返回叶子结点
    return leaf

## 8. 递归地创建决策树
递归的创建决策树  
递归算法终止的三个条件：
1. 如果结点内所有的样本的标记都相同，该结点就不需要再继续划分，直接做叶子结点即可
2. 如果结点所有的特征都已经在之前使用过了，在当前结点无剩余特征可供划分样本，该结点直接做叶子结点
3. 如果当前结点的深度已经达到了我们限制的树的最大深度，直接做叶子结点

**这里需要填写七个部分**

In [33]:
def decision_tree_create(data, features, target, criterion = 'gini', current_depth = 0, max_depth = 10, annotate = False):
    '''
    Parameter:
    ----------
    data: pd.DataFrame, 数据

    features: iterable, 特征组成的可迭代对象，比如一个list

    target: str, 标记的名字

    criterion: 'str', 特征划分方法，只支持三种：'information_gain', 'gain_ratio', 'gini'

    current_depth: int, 当前深度，递归的时候需要记录

    max_depth: int, 树的最大深度，我们设定的树的最大深度，达到最大深度需要终止递归

    Returns:
    ----------
    dict, dict['is_leaf']          : False, 当前顶点不是叶子结点
          dict['prediction']       : None, 不是叶子结点就没有预测值
          dict['splitting_feature']: splitting_feature, 当前结点是使用哪个特征进行划分的
          dict['left']             : dict
          dict['right']            : dict
    '''
    
    if criterion not in ['information_gain', 'gain_ratio', 'gini']:
        raise Exception("传入的criterion不合规!", criterion)
    
    # 复制一份特征，存储起来，每使用一个特征进行划分，我们就删除一个
    remaining_features = features[:]
    
    # 取出标记值
    target_values = data[target]
    print("-" * 50)
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))

    # 终止条件1
    # 如果当前结点内所有样本同属一类，即这个结点中，各类别样本数最小的那个等于0
    # 使用前面写的intermediate_node_num_mistakes来完成这个判断
    if intermediate_node_num_mistakes(target_values) == 0:                              # YOUR CODE HERE
        print("Stopping condition 1 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件2
    # 如果已经没有剩余的特征可供分割，即remaining_features为空
    
    if len(remaining_features) == 0:                                                    # YOUR CODE HERE
        print("Stopping condition 2 reached.")
        return create_leaf(target_values)   # 创建叶子结点
    
    # 终止条件3
    # 如果已经到达了我们要求的最大深度，即当前深度达到了最大深度
    
    if current_depth > max_depth:
        # YOUR CODE HERE
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)   # 创建叶子结点

    # 找到最优划分特征
    # 使用best_splitting_feature这个函数
    
    splitting_feature = best_splitting_feature(data, remaining_features, target, criterion, annotate)                                                                # YOUR CODE HERE
    
    # 使用我们找到的最优特征将数据划分成两份
    # 左子树的数据
    left_split = data[data[splitting_feature] == 0]
    
    # 右子树的数据
    right_split = data[data[splitting_feature] == 1]                                    # YOUR CODE HERE
    
    # 现在已经完成划分，我们要从剩余特征中删除掉当前这个特征
    remaining_features.remove(splitting_feature)
    
    # 打印当前划分使用的特征，打印左子树样本个数，右子树样本个数
    print("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # 如果使用当前的特征，将所有的样本都划分到一棵子树中，那么就直接将这棵子树变成叶子结点
    # 判断左子树是不是“完美”的
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    
    # 判断右子树是不是“完美”的
    if len(right_split) == len(data):
        print("Creating right node.")
        return create_leaf(right_split[target])                                          # YOUR CODE HERE

    # 递归地创建左子树
    left_tree = decision_tree_create(left_split, remaining_features, target, criterion, current_depth + 1, max_depth, annotate)
    
    # 递归地创建右子树
    
    right_tree = decision_tree_create(right_split, remaining_features, target, criterion, current_depth + 1, max_depth, annotate)  # YOUR CODE HERE

    # 返回树的非叶子结点
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

训练一个模型

In [34]:
my_decision_tree = decision_tree_create(train_data, one_hot_features, target, 'gini', max_depth = 6, annotate = False)

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature emp_length_5 years. (67854, 5710)
--------------------------------------------------
Subtree, depth = 1 (67854 data points).
Split on feature emp_length_< 1 year. (61228, 6626)
--------------------------------------------------
Subtree, depth = 2 (61228 data points).
Split on feature home_ownership_OWN. (56181, 5047)
--------------------------------------------------
Subtree, depth = 3 (56181 data points).
Split on feature emp_length_4 years. (51399, 4782)
--------------------------------------------------
Subtree, depth = 4 (51399 data points).
Split on feature emp_length_9 years. (48916, 2483)
--------------------------------------------------
Subtree, depth = 5 (48916 data points).
Split on feature emp_length_8 years. (45939, 2977)
--------------------------------------------------
Subtree, depth = 6 (45939 data points).
Split on feature emp_length_3 years. (40221, 5718)
-----

现在，模型就训练好了

## 9. 预测

接下来我们需要完成预测函数

In [35]:
def classify(tree, x, annotate = False):
    '''
    递归的进行预测，一次只能预测一个样本
    
    Parameters
    ----------
    tree: dict
    
    x: pd.Series，待预测的样本
    
    annotate： boolean, 是否显示注释
    
    Returns
    ----------
    返回预测的标记
    '''
    if tree['is_leaf']:
        if annotate:
            print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

我们取测试集第一个样本来测试

In [36]:
test_sample = test_data.iloc[0]
print(test_sample)

safe_loans                 1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    0
grade_E                    1
grade_F                    0
grade_G                    0
term_ 36 months            1
term_ 60 months            0
home_ownership_MORTGAGE    1
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        0
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
Name: 37225, dtype: int64


In [37]:
print('True class: %s ' % (test_sample['safe_loans']))
print('Predicted class: %s ' % classify(my_decision_tree, test_sample))

True class: 1 
Predicted class: 1 


打印出使用决策树判断的过程

In [38]:
classify(my_decision_tree, test_sample, annotate=True)

Split on emp_length_5 years = 0
Split on emp_length_< 1 year = 0
Split on home_ownership_OWN = 0
Split on emp_length_4 years = 0
Split on emp_length_9 years = 0
Split on emp_length_8 years = 0
Split on emp_length_3 years = 0
At leaf, predicting 1


1

## 10. 在测试集上对我们的模型进行评估

In [39]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

先来编写一个批量预测的函数，传入的是整个测试集那样的pd.DataFrame，这个函数返回一个np.ndarray，存储模型的预测结果  
**这里需要填写一个部分**

In [40]:
def predict(tree, data):
    '''
    按行遍历data，对每个样本进行预测，将值存在prediction中，最后返回np.ndarray
    
    Parameter
    ----------
    tree: dict, 模型
    
    data: pd.DataFrame, 数据
    
    Returns
    ----------
    predictions：np.ndarray, 模型对这些样本的预测结果
    '''
    predictions = np.zeros(len(data)) # 长度和data一样
    
    # YOUR CODE HERE
    for i in range(len(predictions)):
        predictions[i] = classify(tree, data.iloc[i])
    
    return predictions

## 11. 请你计算使用不同评价指标得到模型的四项指标的值，填写在下方表格内
**树的最大深度为6**

### gini

In [41]:
# YOUR CODE HERE
my_decision_tree_gini = decision_tree_create(train_data, one_hot_features, target, 'gini', max_depth = 6, annotate = False)
predictions = predict(my_decision_tree, test_data)
print("model accur is {}".format(accuracy_score(test_data[target], predictions)))
print("model precision is {}".format(precision_score(test_data[target], predictions)))
print("model recall is {}".format(recall_score(test_data[target], predictions)))
print("model f1 is {}".format(f1_score(test_data[target], predictions)))

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature emp_length_5 years. (67854, 5710)
--------------------------------------------------
Subtree, depth = 1 (67854 data points).
Split on feature emp_length_< 1 year. (61228, 6626)
--------------------------------------------------
Subtree, depth = 2 (61228 data points).
Split on feature home_ownership_OWN. (56181, 5047)
--------------------------------------------------
Subtree, depth = 3 (56181 data points).
Split on feature emp_length_4 years. (51399, 4782)
--------------------------------------------------
Subtree, depth = 4 (51399 data points).
Split on feature emp_length_9 years. (48916, 2483)
--------------------------------------------------
Subtree, depth = 5 (48916 data points).
Split on feature emp_length_8 years. (45939, 2977)
--------------------------------------------------
Subtree, depth = 6 (45939 data points).
Split on feature emp_length_3 years. (40221, 5718)
-----

### gain_ratio

In [42]:
# YOUR CODE HERE
my_decision_tree_gain_ratio = decision_tree_create(train_data, one_hot_features, target, 'gain_ratio', max_depth = 6, annotate = False)
predictions = predict(my_decision_tree, test_data)
print("model accur is {}".format(accuracy_score(test_data[target], predictions)))
print("model precision is {}".format(precision_score(test_data[target], predictions)))
print("model recall is {}".format(recall_score(test_data[target], predictions)))
print("model f1 is {}".format(f1_score(test_data[target], predictions)))

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_F. (71229, 2335)
--------------------------------------------------
Subtree, depth = 1 (71229 data points).
Split on feature grade_A. (57869, 13360)
--------------------------------------------------
Subtree, depth = 2 (57869 data points).
Split on feature grade_G. (57232, 637)
--------------------------------------------------
Subtree, depth = 3 (57232 data points).
Split on feature grade_E. (51828, 5404)
--------------------------------------------------
Subtree, depth = 4 (51828 data points).
Split on feature grade_D. (40326, 11502)
--------------------------------------------------
Subtree, depth = 5 (40326 data points).
Split on feature term_ 60 months. (34566, 5760)
--------------------------------------------------
Subtree, depth = 6 (34566 data points).
Split on feature home_ownership_OTHER. (34514, 52)
--------------------------------------------------
Subtree, dep

Split on feature emp_length_2 years. (6702, 569)
--------------------------------------------------
Subtree, depth = 5 (6702 data points).
Split on feature emp_length_4 years. (6234, 468)
--------------------------------------------------
Subtree, depth = 6 (6234 data points).
Split on feature emp_length_1 year. (5880, 354)
--------------------------------------------------
Subtree, depth = 7 (5880 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 7 (354 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (468 data points).
Split on feature grade_B. (468, 0)
Creating leaf node.
--------------------------------------------------
Subtree, depth = 5 (569 data points).
Split on feature grade_B. (569, 0)
Creating leaf node.
--------------------------------------------------
Subtree, depth = 3 (259 data points).
Split on feature emp_length_9 years.

Subtree, depth = 5 (13 data points).
Split on feature grade_A. (13, 0)
Creating leaf node.
--------------------------------------------------
Subtree, depth = 4 (4 data points).
Split on feature grade_A. (4, 0)
Creating leaf node.
model accur is 0.8122463960198193
model precision is 0.8122463960198193
model recall is 1.0
model f1 is 0.8963973086703121


### information_gain

In [43]:
# YOUR CODE HERE
my_decision_tree_info = decision_tree_create(train_data, one_hot_features, target, 'information_gain', max_depth = 6, annotate = False)
predictions = predict(my_decision_tree, test_data)
print("model accur is {}".format(accuracy_score(test_data[target], predictions)))
print("model precision is {}".format(precision_score(test_data[target], predictions)))
print("model recall is {}".format(recall_score(test_data[target], predictions)))
print("model f1 is {}".format(f1_score(test_data[target], predictions)))

--------------------------------------------------
Subtree, depth = 0 (73564 data points).
Split on feature grade_A. (60204, 13360)
--------------------------------------------------
Subtree, depth = 1 (60204 data points).
Split on feature grade_B. (37768, 22436)
--------------------------------------------------
Subtree, depth = 2 (37768 data points).
Split on feature grade_C. (19878, 17890)
--------------------------------------------------
Subtree, depth = 3 (19878 data points).
Split on feature term_ 36 months. (8812, 11066)
--------------------------------------------------
Subtree, depth = 4 (8812 data points).
Split on feature home_ownership_RENT. (5438, 3374)
--------------------------------------------------
Subtree, depth = 5 (5438 data points).
Split on feature grade_D. (3299, 2139)
--------------------------------------------------
Subtree, depth = 6 (3299 data points).
Split on feature grade_E. (1319, 1980)
--------------------------------------------------
Subtree, depth 

Split on feature home_ownership_MORTGAGE. (10775, 9727)
--------------------------------------------------
Subtree, depth = 4 (10775 data points).
Split on feature emp_length_1 year. (9785, 990)
--------------------------------------------------
Subtree, depth = 5 (9785 data points).
Split on feature home_ownership_OTHER. (9754, 31)
--------------------------------------------------
Subtree, depth = 6 (9754 data points).
Split on feature emp_length_2 years. (8457, 1297)
--------------------------------------------------
Subtree, depth = 7 (8457 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 7 (1297 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (31 data points).
Split on feature emp_length_< 1 year. (23, 8)
--------------------------------------------------
Subtree, depth = 7 (23 data points).
Reached maximum depth. Stopping for now.


Split on feature emp_length_2 years. (6879, 581)
--------------------------------------------------
Subtree, depth = 3 (6879 data points).
Split on feature emp_length_4 years. (6397, 482)
--------------------------------------------------
Subtree, depth = 4 (6397 data points).
Split on feature emp_length_1 year. (6036, 361)
--------------------------------------------------
Subtree, depth = 5 (6036 data points).
Split on feature emp_length_3 years. (5475, 561)
--------------------------------------------------
Subtree, depth = 6 (5475 data points).
Split on feature emp_length_8 years. (5113, 362)
--------------------------------------------------
Subtree, depth = 7 (5113 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 7 (362 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------
Subtree, depth = 6 (561 data points).
Split on feature term_ 36 months. (16, 545)
--

树的最大深度为6  

###### 双击此处编写

划分标准|精度|查准率|查全率|F1
-|-|-|-|-
信息增益|0.8121648349407663|0.8122565514428469|0.9998493786870842|0.8963429728817373
信息增益率|0.8117774198152642|0.8127580427584515|0.998242751349316|0.8960016223341333
基尼指数|0.8122463960198193|0.8122463960198193|1.0|0.8963973086703121

## 12. 扩展：使用Echarts绘制决策树

我们可以使用echarts绘制出我们训练的决策树，这时候可以利用pyecharts这个库
[pyecharts](http://pyecharts.org/#/)  
pyecharts可以与jupyter notebook无缝衔接，直接在notebook中绘制图表。
**提醒：pyecharts还未支持jupyter lab**

### 注：由于上方的方案不再适用于新版的pyecharts，绘画代码有所修改，因此代码进行了一定的改动

In [44]:
# 导入树形图
import json
import os

from pyecharts import options as opts
from pyecharts.charts import Page, Tree

echarts中的树形图要求我们提供一组这样的数据  
```
  [
      {
          value: 1212,    # 数值
          # 子节点
          children: [
              {
                  # 子节点数值
                  value: 2323,
                  # 子节点名
                  name: 'description of this node',
                  children: [...],
              },
              {
                  value: 4545,
                  name: 'description of this node',
                  children: [
                      {
                          value: 5656,
                          name: 'description of this node',
                          children: [...]
                      },
                      ...
                  ]
              }
          ]
      },
      ...
  ]
```
关于pyecharts中的树形图的文档地址:[pyecharts Tree](http://pyecharts.org/#/zh-cn/charts?id=tree%EF%BC%88%E6%A0%91%E5%9B%BE%EF%BC%89)

其实和我们训练得到的树结构类似，只不过每个结点有个"name"属性，表示这个结点的名字，"value"表示它的值，"children"是一个list，里面还有这样的dict，我们可以写一个递归的函数完成这种数据的生成

In [45]:
def generate_echarts_data(tree):
    
    # 当前顶点的dict
    value = dict()
    
    # 如果传入的tree已经是叶子结点了
    if tree['is_leaf'] == True:
        
        # 它的value就设置为预测的标记
        value['value'] = tree['prediction']
        
        # 它的名字就叫"label: 标记"
        value['name'] = 'label: %s'%(tree['prediction'])
        value['children']='1'
        # 直接返回这个dict即可
        return value
    
    # 如果传入的tree不是叶子结点，名字就叫当前这个顶点的划分特征，子树是一个list
    # 分别增加左子树和右子树到children中
    value['children'] = [generate_echarts_data(tree['left']), generate_echarts_data(tree['right'])]
    value['name'] = tree['splitting_feature']

    return value

In [46]:
data = generate_echarts_data(my_decision_tree_gini)

使用下面的代码进行绘制，绘制完成后，树的结点是可以点击的，点击后会展开它的子树

In [47]:
# #tree = Tree(width=800, height=400)
# tree=Tree()
# tree.add("",
#          [data],
#          collapse_interval=5,
#          pos_top="15%",
#          pos_right="20%",
#          symbol = 'rect',
#          symbol_size = 20,
#         ).set_global_opts(title_opts=opts.TitleOpts(title="决策树"))
# tree.render_notebook()
# #tree
tree = Tree()
tree.add("", 
         [data],
         collapse_interval=5,
         pos_top="15%",
         pos_right="20%",
         symbol = 'rect',
         symbol_size = 20,).set_global_opts(title_opts=opts.TitleOpts(title="决策树"))
tree.render_notebook()

In [48]:
train_data[train_data['emp_length_5 years']==0][target].value_counts()

 1    54997
-1    12857
Name: safe_loans, dtype: int64

### 注：对于此图有一解释，可以看到，'emp_length_5 years'为0的时候，将90%的均为1，所以会出现如上的节点情况

## 选做：绘制其他的决策树

### gini

In [49]:
# YOUR CODE HERE
data = generate_echarts_data(my_decision_tree_gini)
tree = Tree()
tree.add("", 
         [data],
         collapse_interval=5,
         pos_top="15%",
         pos_right="20%",
         symbol = 'rect',
         symbol_size = 20,).set_global_opts(title_opts=opts.TitleOpts(title="gini"))
tree.render_notebook()

### information_gain

In [50]:
# YOUR CODE HERE
data = generate_echarts_data(my_decision_tree_info)
tree = Tree()
tree.add("", 
         [data],
         collapse_interval=5,
         pos_top="15%",
         pos_right="20%",
         symbol = 'rect',
         symbol_size = 20,).set_global_opts(title_opts=opts.TitleOpts(title="information gain"))
tree.render_notebook()

### gain ratio

In [51]:
# YOUR CODE HERE
data = generate_echarts_data(my_decision_tree_gain_ratio)
tree = Tree()
tree.add("", 
         [data],
         collapse_interval=5,
         pos_top="15%",
         pos_right="20%",
         symbol = 'rect',
         symbol_size = 20,).set_global_opts(title_opts=opts.TitleOpts(title="information gain"))
tree.render_notebook()